In [ ]:
# !pip install snscrape tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 2.1 MB/s eta 0:00:00


In [1]:
from dataclasses import fields
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

pd.set_option('display.max_columns', 80)

In [2]:
# Começamos filtrando todos os tweets de um usuario, dentro do período desejado
username = 'ErikakHilton'  # example user
since = '2022-10-01'
until = '2022-10-02'

query = f'from:{username} since:{since} until:{until}'
user_scrapping_results = sntwitter.TwitterSearchScraper(query)

tweet_ids = []
for tweet in tqdm(user_scrapping_results.get_items()):
    tweet_ids.append(tweet.id)

print(len(tweet_ids))

225it [00:09, 23.81it/s]

225


In [3]:
# Vamos iterar entre os id's para pegar os dados dos tweets e suas respostas
# Esta etapa pode demorar um pouco
tweets_and_replies = []

for tweet_id in tqdm(tweet_ids):
    tweet_scrapper = sntwitter.TwitterTweetScraper(tweet_id, mode=sntwitter.TwitterTweetScraperMode.SCROLL)
    for tweet in tweet_scrapper.get_items():
        tweets_and_replies.append(tweet)
        
print(len(tweets_and_replies))

100%|██████████████████████████████████████████████████████████████████████████████████████| 225/225 [03:27<00:00,  1.08it/s]

1177


In [5]:
# Convertendo as informações para DataFrame
tweet_fields = [f.name for f in fields(tweet)]
tweets_data = [[getattr(t, field) for field in tweet_fields] for t in tweets_and_replies]
tweets_df = pd.DataFrame(tweets_data, columns=tweet_fields)

print(tweets_df.shape)
tweets_df.head()

(1177, 29)


,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/ariranhita/status/15763556...,2022-10-01 23:37:39+00:00,se São Paulo eleger Erika Hilton amanhã eu pro...,se São Paulo eleger Erika Hilton amanhã eu pro...,1576355695594995712,https://twitter.com/ariranhita,53,132,4678,34,1576355695594995712,pt,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
1,https://twitter.com/ErikakHilton/status/157635...,2022-10-01 23:43:25+00:00,@ariranhita HAHAHAHHAHAH 💜💜💜 \n\nE Kiwi com ca...,@ariranhita HAHAHAHHAHAH 💜💜💜 \n\nE Kiwi com ca...,1576357145855262720,https://twitter.com/ErikakHilton,2,1,421,0,1576355695594995712,it,None,None,None,None,None,None,None,1.576356e+18,https://twitter.com/ariranhita,[https://twitter.com/ariranhita],None,None,None,None,None,None,None
2,https://twitter.com/ariranhita/status/15763580...,2022-10-01 23:46:55+00:00,@ErikakHilton ah mas o kiwi nao comsigo abrir ...,@ErikakHilton ah mas o kiwi nao comsigo abrir ...,1576358026457382912,https://twitter.com/ariranhita,3,1,203,5,1576355695594995712,pt,None,None,None,None,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,None,None,1.576357e+18,https://twitter.com/ErikakHilton,[https://twitter.com/ErikakHilton],None,None,None,None,None,None,None
3,https://twitter.com/JulianeRFreitas/status/157...,2022-10-01 23:45:34+00:00,@ErikakHilton @ariranhita Se a Erika for eleit...,@ErikakHilton @ariranhita Se a Erika for eleit...,1576357687084077056,https://twitter.com/JulianeRFreitas,0,0,6,0,1576355695594995712,pt,None,None,None,None,None,None,None,1.576357e+18,https://twitter.com/ErikakHilton,"[https://twitter.com/ErikakHilton, https://twi...",None,None,None,None,None,None,None
4,https://twitter.com/ErikakHilton/status/157631...,2022-10-01 21:03:15+00:00,🔴🔴🔴 IPEC: Lula com 51% dos votos válidos\n\nVa...,🔴🔴🔴 IPEC: Lula com 51% dos votos válidos\n\nVa...,1576316836383113217,https://twitter.com/ErikakHilton,30,154,1510,13,1576316836383113217,pt,None,None,None,None,None,None,None,NaN,None,None,None,None,"[LulaNo1ºTurno, LulaPresidente]",None,None,None,None


In [6]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   url               1177 non-null   object             
 1   date              1177 non-null   datetime64[ns, UTC]
 2   rawContent        1177 non-null   object             
 3   renderedContent   1177 non-null   object             
 4   id                1177 non-null   int64              
 5   user              1177 non-null   object             
 6   replyCount        1177 non-null   int64              
 7   retweetCount      1177 non-null   int64              
 8   likeCount         1177 non-null   int64              
 9   quoteCount        1177 non-null   int64              
 10  conversationId    1177 non-null   int64              
 11  lang              1177 non-null   object             
 12  source            0 non-null      object             
 13  sou

In [7]:
tweets_df['user'][0]

User(username='ariranhita', id=1324085385245446148, displayname='ig: ariaritamusic', rawDescription='ela/elu • professora de alaúde e amante de gatinhos • contato: ariaritamusic@gmail.com ☸\U0001faac', renderedDescription='ela/elu • professora de alaúde e amante de gatinhos • contato: ariaritamusic@gmail.com ☸\U0001faac', descriptionLinks=None, verified=False, created=datetime.datetime(2020, 11, 4, 20, 26, 4, tzinfo=datetime.timezone.utc), followersCount=83962, friendsCount=572, statusesCount=47286, favouritesCount=106739, listedCount=104, mediaCount=5475, location='Besançon, France', protected=False, link=TextLink(text='ariarita.com', url='http://ariarita.com', tcourl='https://t.co/oIS8ztbybc', indices=(0, 23)), profileImageUrl='https://pbs.twimg.com/profile_images/1573466338957344768/JcFdVEsl_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/1324085385245446148/1627555666', label=None)